In [ ]:
!pip install geopandas
!pip install geopy
!pip install folium

     |████████████████████████████████| 972kB 4.0MB/s 
     |████████████████████████████████| 6.5MB 41.5MB/s 
     |████████████████████████████████| 14.8MB 283kB/s 


In [ ]:
import folium 
import csv
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
df = pd.read_csv("noida_wards.csv", encoding= 'unicode_escape')

print(df)

        Sectors
0      Sector 1
1      Sector 2
2      Sector 3
3      Sector 4
4      Sector 5
..          ...
158  Sector 164
159  Sector 165
160  Sector 166
161  Sector 167
162  Sector 168

[163 rows x 1 columns]


In [ ]:
df['Country'] = 'India'
df.head()

,Sectors,Country
0,Sector 1,India
1,Sector 2,India
2,Sector 3,India
3,Sector 4,India
4,Sector 5,India


In [ ]:
df['State'] = 'Noida'
df.head()

,Sectors,Country,State
0,Sector 1,India,Noida
1,Sector 2,India,Noida
2,Sector 3,India,Noida
3,Sector 4,India,Noida
4,Sector 5,India,Noida


In [ ]:
df["location"] = df['Sectors'] +", " + df['State'] +", " + df['Country']
df.head()

,Sectors,Country,State,location
0,Sector 1,India,Noida,"Sector 1, Noida, India"
1,Sector 2,India,Noida,"Sector 2, Noida, India"
2,Sector 3,India,Noida,"Sector 3, Noida, India"
3,Sector 4,India,Noida,"Sector 4, Noida, India"
4,Sector 5,India,Noida,"Sector 5, Noida, India"


In [ ]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
import numpy as np
   
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = []       
  
# function to find the coordinate 
# of a given city  
def findGeocode(location): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="myGeocoder")
        
        return geolocator.geocode(location) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(location) 
# each value from city column 
# will be fetched and sent to 
# function find_geocode    
for i in (df["location"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 

In [ ]:
df["Longitude"] = longitude 
df["Latitude"] = latitude 

df

,Sectors,Country,State,location,Longitude,Latitude
0,Sector 1,India,Noida,"Sector 1, Noida, India",77.391073,28.535633
1,Sector 2,India,Noida,"Sector 2, Noida, India",77.391073,28.535633
2,Sector 3,India,Noida,"Sector 3, Noida, India",77.391073,28.535633
3,Sector 4,India,Noida,"Sector 4, Noida, India",77.391073,28.535633
4,Sector 5,India,Noida,"Sector 5, Noida, India",77.391073,28.535633
...,...,...,...,...,...,...
158,Sector 164,India,Noida,"Sector 164, Noida, India",77.327393,28.586853
159,Sector 165,India,Noida,"Sector 165, Noida, India",77.327393,28.586853
160,Sector 166,India,Noida,"Sector 166, Noida, India",77.327393,28.586853
161,Sector 167,India,Noida,"Sector 167, Noida, India",77.327393,28.586853


In [ ]:
df.to_csv("noida.csv")

In [ ]:
map_hooray = folium.Map(location=[28.590001, 77.350005], zoom_start = 13) 

# Ensure you're handing it floats
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray

In [ ]:
df['Latitude']= df['Latitude'].fillna(0)
df['Longitude']= df['Longitude'].fillna(0)
df.head()

,Sectors,Country,State,location,Longitude,Latitude
0,Sector 1,India,Noida,"Sector 1, Noida, India",77.391073,28.535633
1,Sector 2,India,Noida,"Sector 2, Noida, India",77.391073,28.535633
2,Sector 3,India,Noida,"Sector 3, Noida, India",77.391073,28.535633
3,Sector 4,India,Noida,"Sector 4, Noida, India",77.391073,28.535633
4,Sector 5,India,Noida,"Sector 5, Noida, India",77.391073,28.535633


In [ ]:
latitude = 28.590001
longitude = 77.350005
faridabad_map = folium.Map(location=[latitude, longitude], zoom_start=13)

In [ ]:
for lat, lon, ward in zip(df['Latitude'], df['Longitude'], df['Sectors']):
    folium.CircleMarker(
        [lat, lon],
        popup = ('Ward: ' + str(ward).capitalize()
                ),
        color='blue',
        key_on = ward,
        threshold_scale=[0,1,2,3],
        fill=True,
        fill_opacity=0.7
        ).add_to(faridabad_map)
faridabad_map